In [1]:
import pandas as pd
import pyinaturalist as inat

In [2]:
# get geogenters for all parks

parks = {
    'tucker': (37.667, -77.887, 5),
    'hiddenrock': (37.703, -77.872, 5),
    'leakesmill': (37.705, -77.958, 10),
    'matthews': (37.787, -78.040, 5),
}

In [3]:
res = {}
for park in parks:
    res[park] = pd.json_normalize(inat.get_observation_species_counts(lat=parks[park][0], 
                                                                      lng=parks[park][1], 
                                                                      radius=parks[park][2], 
                                                                      verifiable=True,)['results'])
    normer = pd.json_normalize(inat.get_observation_species_counts(taxon_id=res[park]['taxon.id'].to_list(),
                                                                   verifiable=True,)['results'])
    res[park]['normalizer'] = res[park]['taxon.id'].map(normer.set_index('taxon.id')['count'])
    res[park]['sorter'] = res[park]['count']/res[park]['normalizer']
    res[park].sort_values('sorter',ascending=False,inplace=True)

In [4]:
for park in parks:
    print(f"{park}: {res[park].shape}")

tucker: (500, 42)
hiddenrock: (500, 42)
leakesmill: (500, 42)
matthews: (206, 42)


In [6]:
res['tucker'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
365,8,Heliocheilus lupatus,Spotted Straw Moth,http://en.wikipedia.org/wiki/Heliocheilus_lupatus
442,7,Asarum virginicum,Virginia heartleaf,None
369,8,Lampropeltis rhombomaculata,Mole Kingsnake,https://en.wikipedia.org/wiki/Lampropeltis_cal...
0,107,Ambystoma opacum,Marbled Salamander,http://en.wikipedia.org/wiki/Marbled_salamander
270,10,Nemophila aphylla,smallflower baby blue eyes,http://en.wikipedia.org/wiki/Nemophila_aphylla
271,10,Brachynemurus abdominalis,Bellied Antlion,http://en.wikipedia.org/wiki/Brachynemurus_abd...
12,42,Hetaerina titia,Smoky Rubyspot,None
40,27,Erpetogomphus designatus,Eastern Ringtail,http://en.wikipedia.org/wiki/Erpetogomphus_des...
159,15,Prionus imbricornis,Tile-horned Prionus,http://en.wikipedia.org/wiki/Prionus_imbricornis
103,19,Ornithogalum nutans,Drooping Star-of-Bethlehem,https://en.wikipedia.org/wiki/Ornithogalum_nutans


In [7]:
res['leakesmill'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
451,8,Heliocheilus lupatus,Spotted Straw Moth,http://en.wikipedia.org/wiki/Heliocheilus_lupatus
265,12,Omphalocera munroei,Pawpaw Webworm Moth,http://en.wikipedia.org/wiki/Omphalocera_munroei
457,8,Asarum virginicum,Virginia heartleaf,None
262,12,Nemophila aphylla,smallflower baby blue eyes,http://en.wikipedia.org/wiki/Nemophila_aphylla
0,113,Ambystoma opacum,Marbled Salamander,http://en.wikipedia.org/wiki/Marbled_salamander
455,8,Lampropeltis rhombomaculata,Mole Kingsnake,https://en.wikipedia.org/wiki/Lampropeltis_cal...
11,49,Hetaerina titia,Smoky Rubyspot,None
345,10,Brachynemurus abdominalis,Bellied Antlion,http://en.wikipedia.org/wiki/Brachynemurus_abd...
143,19,Prionus imbricornis,Tile-horned Prionus,http://en.wikipedia.org/wiki/Prionus_imbricornis
98,23,Ornithogalum nutans,Drooping Star-of-Bethlehem,https://en.wikipedia.org/wiki/Ornithogalum_nutans


In [8]:
res['hiddenrock'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
3,32,Hetaerina titia,Smoky Rubyspot,None
21,18,Erpetogomphus designatus,Eastern Ringtail,http://en.wikipedia.org/wiki/Erpetogomphus_des...
215,5,Desmocerus palliatus,Eastern Elderberry Borer,http://en.wikipedia.org/wiki/Desmocerus_palliatus
108,9,Gomphurus vastus,Cobra Clubtail,https://en.wikipedia.org/wiki/Gomphurus_vastus
7,27,Acris crepitans,Northern Cricket Frog,http://en.wikipedia.org/wiki/Northern_cricket_...
473,3,Thelephora vialis,NaN,None
206,6,Asarum minus,little heartleaf,None
18,19,Humulus scandens,Japanese Hops,http://en.wikipedia.org/wiki/Humulus_japonicus
464,3,Callizzia amorata,Gray Scoopwing,http://en.wikipedia.org/wiki/Callizzia_amorata
269,5,Oplismenus undulatifolius,wavyleaf basketgrass,http://en.wikipedia.org/wiki/Oplismenus_undula...


In [9]:
res['matthews'][['count', 'taxon.name', 'taxon.preferred_common_name', 'taxon.wikipedia_url']].head(25)

,count,taxon.name,taxon.preferred_common_name,taxon.wikipedia_url
170,1,Angelica venenosa,hairy angelica,http://en.wikipedia.org/wiki/Angelica_venenosa
189,1,Boletus separans,lilac bolete,http://en.wikipedia.org/wiki/Boletus_separans
25,2,Diastrophus nebulosus,Blackberry Knot Gall Wasp,None
171,1,Dichanthelium boscii,Bosc's Witchgrass,None
66,1,Amanita magniverrucata,Pinecone Amanita,https://en.wikipedia.org/wiki/Amanita_magniver...
85,1,Silene antirrhina,sleepy catchfly,http://en.wikipedia.org/wiki/Silene_antirrhina
172,1,Polygala curtissii,Curtiss's milkwort,http://en.wikipedia.org/wiki/Polygala_curtissii
140,1,Murdannia keisak,marsh dewflower,https://en.wikipedia.org/wiki/Murdannia_keisak
190,1,Contumyces rosellus,rosy navel,None
156,1,Carex platyphylla,broad-leaved sedge,None


In [ ]:
# highlight unusual species esp. engagered ones (sum and check trends)

In [ ]:
# highlight unusual or *unusually-increasing* invasives (sum and check trends)